Importing Packages

In [7]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime

from collections import Counter
from wordcloud import WordCloud , STOPWORDS, ImageColorGenerator

Importing Dataset

In [8]:
conversation = "2.txt"

Find Time and Authors

In [9]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+:[0-9]+) (AM|PM|am|pm) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [10]:
def find_author(s):
    s = s.split(':')
    if len(s) == 2:
        return True
    return False

Find Messages

In [11]:
def message(line):
    split_line = line.split(' - ')
    datetime_str = split_line[0]
    date, time = datetime_str.split(', ')
    message = " ".join(split_line[1:])

    if find_author(message):
        split_message = message.split(': ')
        author = split_message[0]
        message = " ".join(split_message[1:])
    else:
        author = None
    return date, time, author, message

Preparing the Dataset

In [12]:
data = []


with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None

    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = message(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

Sentiment Intensity Analyzer Algorithm

In [13]:
df = pd.DataFrame(data,columns=["Date","Time","Author","Message"])
df["Date"] = pd.to_datetime(df["Date"])
data = df.dropna()

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

print(data.head(10))

Empty DataFrame
Columns: [Date, Time, Author, Message, Positive, Negative, Neutral]
Index: []


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Validate te sentiment

In [14]:
x= sum(data["Positive"])
y= sum(data["Negative"])
z= sum(data["Neutral"])


def score(a,b,c):
    if a>b and a>c:
        print("Positive")
    elif b>a and b>c:
        print("Negative")
    else:
        print("Neutral")
score(x,y,z)

Neutral
